In [ ]:
import sys
sys.path.append('..')

In [ ]:
# 引用 helper 函数
from src.utils.kf_momozhen_equipment_analysis_utils import *

### 全局变量

In [ ]:
START_YEAR = 2024
START_MONTH = 4
END_YEAR = 2024
END_MONTH = 5
DATA_FOLDER_PATH = '..\data'


### 数据分析

In [ ]:
# 使用函数加载和合并数据
equity_data = load_and_merge_equity_data(DATA_FOLDER_PATH, START_YEAR, START_MONTH, END_YEAR, END_MONTH)


In [ ]:
# attr_perc_dict, mysterious_count = get_attribute_percentages(equity_data)


In [ ]:
# print(f"神秘属性出现次数: {mysterious_count}")

In [ ]:
# plot_attribute_percentages(attr_perc_dict, '附加物防')

In [ ]:
# plot_attribute_percentages_all(attr_perc_dict)

In [ ]:
plot_total_percentage(equity_data)


In [ ]:
num_initial_missing = 10
num_final_missing = 0

print_missing_total_percentage(equity_data, num_initial_missing, num_final_missing)


In [ ]:
plot_pie_chart(equity_data, 'rarity')

In [ ]:
plot_pie_chart(equity_data, 'name')

In [ ]:
plot_level(equity_data)


In [ ]:
# 将装备按照有无神秘分成两堆
# mysterious_df = equity_data[equity_data['mysterious'] == True]
# non_mysterious_df = equity_data[equity_data['mysterious'] == False]

In [ ]:
attr_check_df = check_equipment_attribute_data(equity_data)

In [ ]:
# 输出结果查看哪些项是不一致的 (False), 显示 有多少个不一致的项, 并显示前 5 个不一致的项
print(attr_check_df['is_consistent'].value_counts())
# print(attr_check_df[attr_check_df['is_consistent'] == False].head())


In [ ]:
# 检查 attr_1, attr_2, attr_3, attr_4 列是否有缺失值
print(equity_data[['attr_1', 'attr_2', 'attr_3', 'attr_4']].isnull().sum())

In [ ]:
plot_attribute_percentages_i(equity_data, 1)

In [ ]:
plot_attribute_percentages_i(equity_data, 2)

In [ ]:
plot_attribute_percentages_i(equity_data, 3)

In [ ]:
plot_attribute_percentages_i(equity_data, 4)

## User Info

In [ ]:
user_df = load_and_merge_user_info(DATA_FOLDER_PATH, START_YEAR, START_MONTH, END_YEAR, END_MONTH)

In [ ]:
plot_user_rank(user_df)

In [ ]:
plot_user_level(user_df)